In [4]:
import os
import argparse
from PIL import Image
from points import NumberPoints, PointLayoutError
import random
from tqdm import tqdm
import logging
import numpy as np

logging.basicConfig(level=logging.INFO)

GENERAL_CONFIG = {
    # so that we can create more images without the names clashing with the previous
    "version_tag": "",
    "colour": "yellow",
    "boundary_width": 5,
    "background_colour": "#000000",  # for gray: #808080
    "yellow": "#fffe04",
    "min_point_radius": 15,
    "max_point_radius": 25,
    "init_size": 512,
    "mode": "RGB",
    "min_point_num": 1,
    "max_point_num": 5,
    "attempts_limit": 5000,
}

config = GENERAL_CONFIG

In [5]:
class TerminalPointLayoutError(ValueError):
    pass


class OneColourImageGenerator:
    def __init__(self, config):
        self.config = config
        self.nmin = self.config["min_point_num"]
        self.nmax = self.config["max_point_num"]
        self.total_area = self.config["total_area"]
        self._check_areas_make_sense()
        self.setup_directories()
        if self.nmin == 0:
            raise ValueError("min_point_num must be at least 1")

    def _check_areas_make_sense(self):
        if self.total_area is not None:
            min_area_max_num = np.pi * self.config["min_point_radius"] ** 2 * self.nmax
            max_area_max_num = np.pi * self.config["max_point_radius"] ** 2 * self.nmax
            if self.total_area < min_area_max_num:
                raise ValueError(
                    f"total_area is too small. It must be at least {min_area_max_num}"
                )
            if self.total_area > max_area_max_num:
                raise ValueError(
                    f"Total_area is very large, please make total area smaller than {max_area_max_num}"
                )
    
    def setup_directories(self):
        os.makedirs(self.config["IMG_DIR"], exist_ok=True)
        [
            os.makedirs(os.path.join(self.config["IMG_DIR"], str(c)), exist_ok=True)
            for c in range(self.nmin, self.nmax + 1)
        ]

    def create_image(self, n):
        img = Image.new(
            self.config["mode"],
            (self.config["init_size"], self.config["init_size"]),
            color=self.config["background_colour"],
        )
        number_points = NumberPoints(
            img,
            self.config["init_size"],
            yellow=self.config["yellow"],
            blue=None,
            min_point_radius=self.config["min_point_radius"],
            max_point_radius=self.config["max_point_radius"],
            attempts_limit=self.config["attempts_limit"],
        )
        point_array = number_points.design_n_points(n, self.config["colour"])

        if self.total_area is not None:
            point_array = number_points.fix_total_area(point_array, self.total_area)
        return number_points.draw_points(point_array)

    def create_and_save(self, n, tag=""):
        v_tag = f"_{config['version_tag']}" if config["version_tag"] is not None else ""
        ac_tag = "_ac" if self.total_area is not None else "" # is it area controlled?
        name = f"img_{n}_{tag}{ac_tag}{v_tag}.png"

        attempts = 0
        while attempts < self.config["attempts_limit"]:
            try:
                self.create_and_save_once(name, n)
                break
            except PointLayoutError as e:
                logging.debug(f"Failed to create image {name} because '{e}' Retrying.")
                attempts += 1

                if attempts == self.config["attempts_limit"]:
                    raise TerminalPointLayoutError(
                        f"""Failed to create image {name} after {attempts} attempts. 
                        Your points are probably too big, or there are too many. 
                        Stopping."""
                    )

    def create_and_save_once(self, name, n):
        img = self.create_image(n)
        img.save(os.path.join(self.config["IMG_DIR"], str(n), name))

    def generate_images(self):

        logging.info(
            f"This will make {self.config['IMAGE_SET_NUM'] * (self.nmax - self.nmin + 1) * 4} images."
        )
        for i in tqdm(range(self.config["IMAGE_SET_NUM"])):
            for n in range(self.nmin, self.nmax + 1):
                self.create_and_save(n, tag=i)


In [6]:
def get_config():
    parser = argparse.ArgumentParser(
        description="Generate images based on number point configurations."
    )
    parser.add_argument(
        "--img_set_num",
        type=int,
        default=100,
        help="Number of image sets to generate.",
    )
    parser.add_argument(
        "--img_dir",
        type=str,
        default="images/extremely_easy",
        help="Directory to save images.",
    )
    parser.add_argument(
        "--total_area",
        type=int,
        default=None,
        help="Total area of the points in the image. Can be None.",
    )

    args = parser.parse_args()

    config = GENERAL_CONFIG | {
        "IMAGE_SET_NUM": args.img_set_num,
        "IMG_DIR": args.img_dir,
        "total_area": args.total_area,
    }
    return config

In [10]:
config = GENERAL_CONFIG | {
    "IMAGE_SET_NUM": 200,
    "IMG_DIR": "../images/one_colour/test",
    "total_area": 4500,
}
random.seed(1714)
image_generator = OneColourImageGenerator(config)
image_generator.generate_images()

INFO:root:This will make 4000 images.
100%|██████████| 200/200 [00:12<00:00, 15.78it/s]


In [13]:
np.pi * 20 ** 2 * 9

11309.733552923255